In [ ]:
!pip install pandas pyarrow duckdb


In [ ]:
import os

os.makedirs('data/bronze', exist_ok=True)
os.makedirs('data/silver', exist_ok=True)
os.makedirs('data/gold', exist_ok=True)

print("Pastas criadas com sucesso!")


Pastas criadas com sucesso!


In [ ]:
import requests
import pandas as pd
import io
import gzip

# URL do brasil.io
url = "https://data.brasil.io/dataset/gastos-deputados/cota_parlamentar.csv.gz"

# headers simulando navegador
headers = {"User-Agent": "Mozilla/5.0"}

# faz download
response = requests.get(url, headers=headers)

if response.status_code == 200:
    compressed_file = io.BytesIO(response.content)
    decompressed_file = gzip.open(compressed_file, mode='rt')
    df_bronze = pd.read_csv(decompressed_file)

    bronze_path = 'data/bronze/gastos_deputados_raw.parquet'
    df_bronze.to_parquet(bronze_path, index=False)
    print(f"Arquivo salvo na camada Bronze: {bronze_path}")
    df_bronze.head()
else:
    print(f"Erro ao baixar: {response.status_code}")





Arquivo salvo na camada Bronze: data/bronze/gastos_deputados_raw.parquet


In [ ]:
# Carrega o Parquet da camada Bronze
df_silver = pd.read_parquet('data/bronze/gastos_deputados_raw.parquet')

# Visualiza colunas para decidir o que limpar
print("Colunas disponíveis:")
print(df_silver.columns)

# --- ETAPAS DE LIMPEZA ---

# Exemplo: colunas que podem ser removidas
colunas_remover = [
    'cnpj_cpf', 'document_id', 'url_documento', 'numero', 'ano',  # você pode ajustar depois
]
df_silver = df_silver.drop(columns=colunas_remover, errors='ignore')

# Tratar valores nulos: substitui valores nulos de valor reembolsado por 0
if 'valor_reembolsado' in df_silver.columns:
    df_silver['valor_reembolsado'] = df_silver['valor_reembolsado'].fillna(0)

# Converte data para datetime (se existir)
if 'data' in df_silver.columns:
    df_silver['data'] = pd.to_datetime(df_silver['data'], errors='coerce')

# Padroniza nomes de parlamentares para caixa alta (opcional)
if 'parlamentar' in df_silver.columns:
    df_silver['parlamentar'] = df_silver['parlamentar'].str.upper()

# Salva camada Silver
silver_path = 'data/silver/gastos_deputados_clean.parquet'
df_silver.to_parquet(silver_path, index=False)

print(f"Arquivo salvo na camada Silver: {silver_path}")
df_silver.head()


Colunas disponíveis:
Index(['codlegislatura', 'datemissao', 'idedocumento', 'idecadastro',
       'indtipodocumento', 'nucarteiraparlamentar', 'nudeputadoid',
       'nulegislatura', 'numano', 'numespecificacaosubcota', 'numlote',
       'nummes', 'numparcela', 'numressarcimento', 'numsubcota', 'sgpartido',
       'sguf', 'txnomeparlamentar', 'txtcnpjcpf', 'txtdescricao',
       'txtdescricaoespecificacao', 'txtfornecedor', 'txtnumero',
       'txtpassageiro', 'txttrecho', 'vlrdocumento', 'vlrglosa', 'vlrliquido',
       'vlrrestituicao'],
      dtype='object')
Arquivo salvo na camada Silver: data/silver/gastos_deputados_clean.parquet


,codlegislatura,datemissao,idedocumento,idecadastro,indtipodocumento,nucarteiraparlamentar,nudeputadoid,nulegislatura,numano,numespecificacaosubcota,...,txtdescricao,txtdescricaoespecificacao,txtfornecedor,txtnumero,txtpassageiro,txttrecho,vlrdocumento,vlrglosa,vlrliquido,vlrrestituicao
0,56,2020-02-01 00:00:00,7007086,NaN,0,NaN,2442,2019,2020,0,...,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,None,CONTENT ASSESSORIA LTDA - ME,432,None,None,998.0,0.0,998.0,NaN
1,56,2020-02-13 00:00:00,7017162,NaN,4,NaN,2442,2019,2020,0,...,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,None,METROPOLE INTERATIVA INFORMATICA LTDA,46,None,None,4920.0,0.0,4920.0,NaN
2,56,2020-02-04 00:00:00,7007754,NaN,4,NaN,2442,2019,2020,0,...,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,None,AFC ALIMENTOS LTDA,306,None,None,511.0,0.0,511.0,NaN
3,56,2020-02-18 00:00:00,7018634,NaN,4,NaN,2442,2019,2020,0,...,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,None,AFC ALIMENTOS LTDA,311,None,None,186.0,0.0,186.0,NaN
4,56,2020-02-11 00:00:00,7011106,NaN,4,NaN,2442,2019,2020,0,...,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,None,ARTESANAL SERVICOS DE ALIMENTACAO E BUFFET EIRELI,23,None,None,740.0,0.0,740.0,NaN


In [ ]:
# Carrega dados limpos
df_silver = pd.read_parquet('data/silver/gastos_deputados_clean.parquet')

# -----------------------------
# 1. Gastos totais por parlamentar
# -----------------------------
gold1 = df_silver.groupby('txnomeparlamentar')['vlrdocumento'].sum().reset_index()
gold1 = gold1.sort_values(by='vlrdocumento', ascending=False)
gold1.to_parquet('data/gold/gastos_por_parlamentar.parquet', index=False)
print("Tabela 1 salva: gastos_por_parlamentar.parquet")

# -----------------------------
# 2. Gastos totais por partido por ano
# -----------------------------
if 'sgpartido' in df_silver.columns and 'datemissao' in df_silver.columns:
    df_silver['ano'] = pd.to_datetime(df_silver['datemissao'], errors='coerce').dt.year
    gold2 = df_silver.groupby(['sgpartido', 'ano'])['vlrdocumento'].sum().reset_index()
    gold2.to_parquet('data/gold/gastos_por_partido_ano.parquet', index=False)
    print("Tabela 2 salva: gastos_por_partido_ano.parquet")
else:
    print("Colunas necessárias não encontradas.")


Tabela 1 salva: gastos_por_parlamentar.parquet
Tabela 2 salva: gastos_por_partido_ano.parquet


In [ ]:
import duckdb

# Conecta ao DuckDB em memória
con = duckdb.connect()

# -----------------------------
# Consulta 1: Top 10 deputados que mais gastaram
# -----------------------------
print("\n🟡 Consulta 1: Top 10 parlamentares por valor gasto")
q1 = """
SELECT txnomeparlamentar, vlrdocumento AS total_gasto
FROM 'data/gold/gastos_por_parlamentar.parquet'
ORDER BY total_gasto DESC
LIMIT 10
"""
print(con.execute(q1).fetchdf())

# -----------------------------
# Consulta 2: Gastos por partido no ano de 2023
# -----------------------------
print("\n🔵 Consulta 2: Gastos por partido em 2023")
q2 = """
SELECT sgpartido, SUM(vlrdocumento) AS total_2023
FROM 'data/gold/gastos_por_partido_ano.parquet'
WHERE ano = 2023
GROUP BY sgpartido
ORDER BY total_2023 DESC
"""
print(con.execute(q2).fetchdf())

# -----------------------------
# Consulta 3: Média de gastos por deputado
# -----------------------------
print("\n🟢 Consulta 3: Média de gastos por parlamentar")
q3 = """
SELECT AVG(vlrdocumento) AS media_gastos
FROM 'data/gold/gastos_por_parlamentar.parquet'
"""
print(con.execute(q3).fetchdf())



🟡 Consulta 1: Top 10 parlamentares por valor gasto
          txnomeparlamentar   total_gasto
0        Wellington Roberto  5.441989e+06
1              Silas Câmara  5.232379e+06
2                Edio Lopes  5.087785e+06
3                 Beto Faro  4.851948e+06
4              Efraim Filho  4.846696e+06
5  José Airton Félix Cirilo  4.842468e+06
6              Cleber Verde  4.831059e+06
7          Elcione Barbalho  4.793567e+06
8            José Guimarães  4.750622e+06
9             Flaviano Melo  4.747061e+06

🔵 Consulta 2: Gastos por partido em 2023
Empty DataFrame
Columns: [sgpartido, total_2023]
Index: []

🟢 Consulta 3: Média de gastos por parlamentar
   media_gastos
0  1.444906e+06


In [ ]:
import pandas as pd

df = pd.read_parquet("data/gold/gastos_por_partido_ano.parquet")
print(df['ano'].unique())


[2009. 2011. 2015. 2016. 2017. 2018. 2019. 2020. 2001. 2008. 2010. 2012.
 2013. 2014. 2099. 2209. 2111. 2000. 2007. 2002. 2005. 2077. 2042. 2201.
 2004. 2006. 2021. 2100. 1959. 2200. 2090.]


In [ ]:
# -----------------------------
# Consulta 2: Gastos por partido de 2015 a 2023
# -----------------------------
print("\n🔵 Consulta 2: Gastos por partido (anos válidos)")
q2 = """
SELECT sgpartido, ano, SUM(vlrdocumento) AS total
FROM 'data/gold/gastos_por_partido_ano.parquet'
WHERE ano BETWEEN 2015 AND 2023
GROUP BY sgpartido, ano
ORDER BY ano, total DESC
"""
print(con.execute(q2).fetchdf())



🔵 Consulta 2: Gastos por partido (anos válidos)
    sgpartido     ano        total
0          PT  2015.0  24581279.96
1          PP  2015.0  22062931.04
2         MDB  2015.0  21869983.60
3        PSDB  2015.0  19724039.83
4         DEM  2015.0  16044756.14
..        ...     ...          ...
180      REDE  2020.0     34579.92
181       SDD  2020.0      2880.14
182    LIDMIN  2020.0      1967.00
183      PMDB  2020.0      1200.00
184       PPS  2021.0        25.00

[185 rows x 3 columns]


In [ ]:
import duckdb

# Conexão com DuckDB
con = duckdb.connect()

# -----------------------------
# Consulta 1: Top 10 parlamentares por valor gasto
# -----------------------------
print("\n🟡 Consulta 1: Top 10 parlamentares")
q1 = """
SELECT txnomeparlamentar, vlrdocumento AS total_gasto
FROM 'data/gold/gastos_por_parlamentar.parquet'
ORDER BY total_gasto DESC
LIMIT 10
"""
print(con.execute(q1).fetchdf())

# -----------------------------
# Consulta 2: Gastos por partido (anos válidos)
# -----------------------------
print("\n🔵 Consulta 2: Gastos por partido (2015–2023)")
q2 = """
SELECT sgpartido, ano, SUM(vlrdocumento) AS total
FROM 'data/gold/gastos_por_partido_ano.parquet'
WHERE ano BETWEEN 2015 AND 2023
GROUP BY sgpartido, ano
ORDER BY ano, total DESC
"""
print(con.execute(q2).fetchdf())

# -----------------------------
# Consulta 3: Média de gastos por parlamentar
# -----------------------------
print("\n🟢 Consulta 3: Média de gastos por parlamentar")
q3 = """
SELECT AVG(vlrdocumento) AS media_gastos
FROM 'data/gold/gastos_por_parlamentar.parquet'
"""
print(con.execute(q3).fetchdf())



🟡 Consulta 1: Top 10 parlamentares
          txnomeparlamentar   total_gasto
0        Wellington Roberto  5.441989e+06
1              Silas Câmara  5.232379e+06
2                Edio Lopes  5.087785e+06
3                 Beto Faro  4.851948e+06
4              Efraim Filho  4.846696e+06
5  José Airton Félix Cirilo  4.842468e+06
6              Cleber Verde  4.831059e+06
7          Elcione Barbalho  4.793567e+06
8            José Guimarães  4.750622e+06
9             Flaviano Melo  4.747061e+06

🔵 Consulta 2: Gastos por partido (2015–2023)
    sgpartido     ano        total
0          PT  2015.0  24581279.96
1          PP  2015.0  22062931.04
2         MDB  2015.0  21869983.60
3        PSDB  2015.0  19724039.83
4         DEM  2015.0  16044756.14
..        ...     ...          ...
180      REDE  2020.0     34579.92
181       SDD  2020.0      2880.14
182    LIDMIN  2020.0      1967.00
183      PMDB  2020.0      1200.00
184       PPS  2021.0        25.00

[185 rows x 3 columns]

🟢 Consulta 

In [ ]:
with open("consultas_duckdb.py", "w") as f:
    f.write("""
import duckdb

con = duckdb.connect()

print("\\n🟡 Consulta 1: Top 10 parlamentares")
q1 = \"""
SELECT txnomeparlamentar, vlrdocumento AS total_gasto
FROM 'data/gold/gastos_por_parlamentar.parquet'
ORDER BY total_gasto DESC
LIMIT 10
\"""
print(con.execute(q1).fetchdf())

print("\\n🔵 Consulta 2: Gastos por partido (2015–2023)")
q2 = \"""
SELECT sgpartido, ano, SUM(vlrdocumento) AS total
FROM 'data/gold/gastos_por_partido_ano.parquet'
WHERE ano BETWEEN 2015 AND 2023
GROUP BY sgpartido, ano
ORDER BY ano, total DESC
\"""
print(con.execute(q2).fetchdf())

print("\\n🟢 Consulta 3: Média de gastos por parlamentar")
q3 = \"""
SELECT AVG(vlrdocumento) AS media_gastos
FROM 'data/gold/gastos_por_parlamentar.parquet'
\"""
print(con.execute(q3).fetchdf())
""")


In [ ]:
from google.colab import files
files.download("consultas_duckdb.py")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import shutil

# Cria estrutura final
os.makedirs("projeto_final/data", exist_ok=True)

# Copia as camadas Bronze, Silver e Gold
shutil.copytree("data/bronze", "projeto_final/data/bronze")
shutil.copytree("data/silver", "projeto_final/data/silver")
shutil.copytree("data/gold", "projeto_final/data/gold")

# Copia o script de consultas
shutil.copy("consultas_duckdb.py", "projeto_final/consultas_duckdb.py")

# Criar README.md
readme_content = """# ETL - Gastos Parlamentares com DuckDB

Projeto acadêmico de Business Intelligence - LIBERTAS

## 🔧 Tecnologias

- Python + Pandas
- Parquet (.parquet)
- DuckDB (consultas SQL)
- Google Colab

## 🧱 Camadas do ETL

- **Bronze**: dados brutos
- **Silver**: dados tratados
- **Gold**: dados agregados para análise

## 📊 Consultas DuckDB

As consultas estão no script `consultas_duckdb.py`, incluindo:

1. Top 10 parlamentares que mais gastaram
2. Gastos por partido (anos válidos)
3. Média de gastos por parlamentar

## 🎥 Apresentação

Link do vídeo (YouTube):
👉 [colar aqui]

## 📁 Estrutura de pastas

data/
├── bronze/
├── silver/
└── gold/

scss
Copiar
"""

with open("projeto_final/README.md", "w") as f:
    f.write(readme_content)

# Compactar tudo
shutil.make_archive("projeto_etl_duckdb", "zip", "projeto_final")

# Download do .zip
from google.colab import files
files.download("projeto_etl_duckdb.zip")



FileExistsError: [Errno 17] File exists: 'projeto_final/data/bronze'

In [ ]:
import shutil
from google.colab import files

# Compacta a pasta existente
shutil.make_archive("projeto_etl_duckdb", "zip", "projeto_final")

# Faz download
files.download("projeto_etl_duckdb.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>